In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
col_names = ["Player", "Team", "Pos", "p/g", "3/g", "r/g", "a/g", "s/g", "b/g", "fg%", "fga/g", "ft%", "fta/g", "to/g"]
proj_df = pd.read_excel('projection_12_10_25.xlsx')
proj_df = proj_df[col_names]
proj_df["Player"] = proj_df["Player"].str.lower()
proj_df

,Player,Team,Pos,p/g,3/g,r/g,a/g,s/g,b/g,fg%,fga/g,ft%,fta/g,to/g
0,nikola jokic,DEN,C,27.45,1.56,12.35,9.72,1.36,0.78,0.565,18.47,0.820,6.10,3.16
1,shai gilgeous-alexander,OKC,PG/SG,31.69,2.00,5.15,6.38,1.72,0.95,0.517,21.25,0.887,8.71,2.42
2,victor wembanyama,SAS,PF/C,24.60,2.95,11.62,4.16,1.13,3.78,0.484,18.43,0.830,4.61,3.19
3,anthony davis,DAL,PF/C,24.79,0.58,11.12,3.87,1.13,2.13,0.532,17.77,0.785,6.75,2.18
4,luka doncic,LAL,PG/SG,28.79,3.57,8.18,7.99,1.48,0.49,0.458,20.81,0.780,7.91,3.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,jaden ivey,DET,PG/SG,16.04,1.74,3.58,3.58,0.82,0.39,0.458,12.53,0.750,3.76,2.64
169,ace bailey,UTA,SF,12.00,1.23,5.42,1.08,0.79,0.72,0.432,10.72,0.700,2.14,1.52
170,kyle kuzma,MIL,SF/PF,13.52,1.31,4.90,2.18,0.47,0.47,0.465,11.22,0.718,2.47,1.84
171,tre johnson,WAS,SG,12.70,1.94,2.29,1.94,0.69,0.14,0.399,11.36,0.870,1.93,1.51


### Descriptive Analysis

### Data Transformation and Modeling

In [3]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [4]:
df_score = proj_df[['fg%', 'ft%', '3/g', 'r/g', 'a/g', 's/g', 'b/g', 'to/g', 'p/g']]
df_attempts = proj_df[['fga/g', 'fta/g']]

scaler = StandardScaler()
mm_scaler = MinMaxScaler()

scaled = scaler.fit_transform(df_score)
att_scaled = mm_scaler.fit_transform(df_attempts)

# Put back into DataFrame
scaled_df = pd.DataFrame(scaled, columns=df_score.columns, index=df_score.index)
att_scaled_df = pd.DataFrame(att_scaled, columns=df_attempts.columns, index=df_attempts.index)

# Combine back with string columns
scaled_score_df = pd.concat([proj_df[['Player', 'Pos', 'Team']], att_scaled_df ,scaled_df], axis=1)

scaled_score_df["to/g"] *= -1

scaled_score_df["fg%"] = scaled_score_df["fga/g"] * scaled_score_df["fg%"]
scaled_score_df["ft%"] = scaled_score_df["fta/g"] * scaled_score_df["ft%"]
scaled_score_df.drop(['fga/g','fta/g'], axis=1, inplace=True)

perc_df = scaled_score_df[['fg%', 'ft%']]
perc_scaled = scaler.fit_transform(perc_df)
perc_scaled_df = pd.DataFrame(perc_scaled, columns=perc_df.columns, index=perc_df.index)
scaled_score_df[['fg%', 'ft%']] = perc_scaled_df

In [5]:
scaled_score_df

,Player,Pos,Team,fg%,ft%,3/g,r/g,a/g,s/g,b/g,to/g,p/g
0,nikola jokic,C,DEN,3.145951,0.735458,-0.174644,2.760155,3.120960,1.182123,0.161298,-1.675678,2.071145
1,shai gilgeous-alexander,PG/SG,OKC,1.664867,3.273051,0.288236,-0.247131,1.397407,2.291901,0.480959,-0.696267,2.848068
2,victor wembanyama,PF/C,SAS,0.245584,0.691784,1.287638,2.455250,0.251813,0.473099,5.802374,-1.715384,1.548921
3,anthony davis,PF/C,DAL,1.878486,-0.041952,-1.205606,2.246410,0.102163,0.473099,2.699782,-0.378621,1.583736
4,luka doncic,PG/SG,LAL,-0.828051,-0.183341,1.939879,1.018436,2.228222,1.552049,-0.384006,-2.297736,2.316682
...,...,...,...,...,...,...,...,...,...,...,...,...
168,jaden ivey,PG/SG,DET,-0.322621,-0.518045,0.014716,-0.902886,-0.047487,-0.482543,-0.572042,-0.987444,-0.019584
169,ace bailey,SF,UTA,-0.642741,-0.652446,-0.521805,-0.134357,-1.337571,-0.575024,0.048476,0.494908,-0.759860
170,kyle kuzma,SF/PF,MIL,-0.118032,-0.611775,-0.437645,-0.351550,-0.769934,-1.561493,-0.421613,0.071379,-0.481340
171,tre johnson,SG,WAS,-1.322107,0.457273,0.225116,-1.441691,-0.893782,-0.883296,-1.042132,0.508143,-0.631594


In [6]:
df_basic_rank = scaled_score_df.reset_index(drop=True)
df_basic_rank["Val"] = df_basic_rank.select_dtypes(include=["number"]).sum(axis=1)
df_basic_rank.sort_values(by='Val', ascending=False, inplace=True)
df_basic_rank.reset_index(drop=True, inplace=True)

In [7]:
df_basic_rank

,Player,Pos,Team,fg%,ft%,3/g,r/g,a/g,s/g,b/g,to/g,p/g,Val
0,nikola jokic,C,DEN,3.145951,0.735458,-0.174644,2.760155,3.120960,1.182123,0.161298,-1.675678,2.071145,11.326767
1,shai gilgeous-alexander,PG/SG,OKC,1.664867,3.273051,0.288236,-0.247131,1.397407,2.291901,0.480959,-0.696267,2.848068,11.301091
2,victor wembanyama,PF/C,SAS,0.245584,0.691784,1.287638,2.455250,0.251813,0.473099,5.802374,-1.715384,1.548921,11.041077
3,anthony davis,PF/C,DAL,1.878486,-0.041952,-1.205606,2.246410,0.102163,0.473099,2.699782,-0.378621,1.583736,7.357497
4,kevin durant,SF/PF,HOU,1.447034,1.825213,1.045677,-0.017407,0.685281,-0.544197,0.819423,-2.085972,2.314850,5.489902
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,ace bailey,SF,UTA,-0.642741,-0.652446,-0.521805,-0.134357,-1.337571,-0.575024,0.048476,0.494908,-0.759860,-4.080421
169,adem bona,PF/C,PHI,0.357600,-0.882729,-1.815767,-0.034114,-1.564626,-1.222395,1.439942,0.786084,-1.595419,-4.531425
170,kyle kuzma,SF/PF,MIL,-0.118032,-0.611775,-0.437645,-0.351550,-0.769934,-1.561493,-0.421613,0.071379,-0.481340,-4.682004
171,tre johnson,SG,WAS,-1.322107,0.457273,0.225116,-1.441691,-0.893782,-0.883296,-1.042132,0.508143,-0.631594,-5.024070


In [8]:
def highlight_group_boundaries(row):
    # row.name gives the row index
    if (row.name + 1) % 8 == 0:  # every 8th row
        return ["border-bottom: 3px solid black"] * len(row)
    else:
        return [""] * len(row)

In [9]:
df_basic_rank[["Player"]][:170].style.apply(highlight_group_boundaries, axis=1)

,Player
0,nikola jokic
1,shai gilgeous-alexander
2,victor wembanyama
3,anthony davis
4,kevin durant
5,luka doncic
6,joel embiid
7,karl-anthony towns
8,evan mobley
9,jalen williams


In [10]:
df_basic_rank["Opt_Val"] = df_basic_rank["Val"] - df_basic_rank[["fg%",	"ft%", "3/g",
                                                             "r/g",	"a/g",	"s/g",
                                                             "b/g",	"to/g",	"p/g"]].min(axis=1)

In [11]:
df_basic_rank.sort_values(by='Opt_Val', ascending=False, inplace=True)
df_basic_rank.reset_index(drop=True, inplace=True)

In [12]:
df_basic_rank[["Player"]][:170].style.apply(highlight_group_boundaries, axis=1)

,Player
0,nikola jokic
1,victor wembanyama
2,shai gilgeous-alexander
3,anthony davis
4,luka doncic
5,giannis antetokounmpo
6,kevin durant
7,joel embiid
8,cade cunningham
9,karl-anthony towns
